In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

####################################################################
#######CONSTANTS:
####################################################################
sampleRate = 128     #per second
windowHalfLength = 8 #on either side of the point of interest (for the moving window)
peakThreshold = 0.8  #peakThreshold*mean value identifies a peak

#low frequency band
lowfBandMin = 0
lowfBandMax = 2
#mid frequency band
midfBandMin = 2
midfBandMax = 4
#high frequency band
highfBandMin = 4
highfBandMax = 6

dummyVariable = 9 #ignore this for now, please.

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This notebook processes the tdcsfog data for the Parkinson's Freezing of Gait Challenge for later use.  

**This notebook processes the tdcsfog data for the Parkinson's Freezing of Gait Challenge and creates new csv files that can then be routed into a Kaggle data set for later use. These files contain new columns which include new features to feed to the neural network.**

# **Functions:**

In [ ]:
#A fxn that computes the root mean abs square value of an array.
def getRMS(inputArray):
    runningSum = 0
    for element in range(0, len(inputArray)):
        runningSum += np.sqrt(np.mean(abs(inputArray*inputArray)))
    return runningSum


#A low pass filter to remove high frequency noise.
def lowPassFilter(kArr, freqArr, cutOffFreq):
    for i in range(0,len(freqArr)):
        if freqArr[i] > cutOffFreq:
            kArr.real[i] = 0;
            kArr.imag[i] = 0;
    return kArr


#A high pass filter to analyze only high frequencies.  
def highPassFilter(kArr, freqArr, cutOffFreq):
    for i in range(0,len(freqArr)):
        if freqArr[i] < cutOffFreq:
            kArr.real[i] = 0;
            kArr.imag[i] = 0;
    return kArr


#A quick FFT where W can be x, y, z accelerations etc.
def quickFFT(inputT, inputW, sampleRate, filterType, cutOff):
    kspaceData = np.fft.rfft(inputW)
    freq = np.fft.rfftfreq(inputT.shape[-1], d=1.0/sampleRate)
    if filterType == "low":
        filteredData = lowPassFilter(kspaceData, freq, cutOff)
    elif filterType == "high":
        filteredData = highPassFilter(kspaceData, freq, cutOff)
    else:
        filteredData = kspaceData
    outputW = np.fft.irfft(filteredData, len(inputW))
    return outputW


#A quick FFT where W can be x, y, z accelerations etc. (returns k-space)
def quickFFT_k(inputT, inputW, sampleRate, filterType, cutOff):
    kspaceData = np.fft.rfft(inputW)
    freq = np.fft.rfftfreq(inputT.shape[-1], d=1.0/sampleRate)
    if filterType == "low":
        filteredData = lowPassFilter(kspaceData, freq, cutOff)
    elif filterType == "high":
        filteredData = highPassFilter(kspaceData, freq, cutOff)
    else:
        filteredData = kspaceData
    return freq, filteredData

# **Read in, Process tdcsfog Data, and Write Files:**

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        tempData = np.loadtxt(dirname+filename, delimiter=',', skiprows=1)
        #Start with this data:
        t = tempData[:,0]
        aVert = tempData[:,1]
        aML = tempData[:,2]
        aAP = tempData[:,3]
        boolStartHes = tempData[:,4]
        boolTurn = tempData[:,5]
        boolWalking = tempData[:,6]
        numPoints = len(t)
        featureList = [[] for _ in range(0, numPoints)]
        #Now process various metrics/features in a moving window fashion:
        #Let's pad the edges with zeros for now.
        ampSumML         = np.zeros(numPoints) # sum of absolute value of (FFT) amplitudes in window (ML)
        ampSumAP         = np.zeros(numPoints) # same, but for AP
        ampSumVert       = np.zeros(numPoints) # same, but for vert
        maxAmpML         = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd ML data
        maxAmpAP         = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd AP data
        maxAmpVert       = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd vert data
        numPeaksML       = np.zeros(numPoints) # number of peaks in accel data (ML) above some threshold
        numPeaksAP       = np.zeros(numPoints) # number of peaks in accel data (AP) above some threshold
        numPeaksVert     = np.zeros(numPoints) # number of peaks in accel data (Vert) above some threshold
        maxAccelAmpML    = np.zeros(numPoints) # abs(max - min accel) for ML accel data
        maxAccelAmpAP    = np.zeros(numPoints) # abs(max - min accel) for AP accel data
        maxAccelAmpVert  = np.zeros(numPoints) # abs(max - min accel) for Vert accel data
        rmsAmpsLowfML    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (ML)
        rmsAmpsLowfAP    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (AP)
        rmsAmpsLowfVert  = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (vert)
        rmsAmpsMidfML    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in mid freq band (ML)
        rmsAmpsMidfAP    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in mid freq band (AP)
        rmsAmpsMidfVert  = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in mid freq band (vert)
        rmsAmpsHighfML   = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (ML)
        rmsAmpsHighfAP   = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (AP)
        rmsAmpsHighfVert = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (vert)
        freqOfMaxAmpML   = np.zeros(numPoints) # frequency of the largest (absolute) magnitude in FFT'd data (ML)
        freqOfMaxAmpAP   = np.zeros(numPoints) # frequency of the largest (absolute) magnitude in FFT'd data (AP)
        freqOfMaxAmpVert = np.zeros(numPoints) # frequency of the largest (absolute) magnitude in FFT'd data (Vert)
        ratio1           = np.zeros(numPoints) # ratio of rms high freq ML to rms low freq AP
        ratio2           = np.zeros(numPoints) # ratio of rms high freq ML to rms high freq AP
        ratio3           = np.zeros(numPoints) # ratio of abs(maxAmpML) to average abs(AP amplitude) (FFT'd)
        ratio4           = np.zeros(numPoints) # ratio of freqOfMaxAmpML/freqOfMaxAmpAP
        ratio5           = np.zeros(numPoints) # ratio of freqOfMaxAmpVert/freqOfMaxAmpAP
        ratio6           = np.zeros(numPoints) # ratio of freqOfMaxAmpML/freqOfMaxAmpVert
        #maybe try odd and even terms (imag and real) to see if there are symmetries to exploit
        for tm in range(windowHalfLength, numPoints-windowHalfLength-1):
            tClip = t[tm-windowHalfLength:tm+windowHalfLength+1]
            aML_Clip = aML[tm-windowHalfLength:tm+windowHalfLength+1]
            aAP_Clip = aAP[tm-windowHalfLength:tm+windowHalfLength+1]
            aVert_Clip = aVert[tm-windowHalfLength:tm+windowHalfLength+1]
            freqML, ampsML = quickFFT_k(tClip, aML_Clip, sampleRate, "none", dummyVariable)
            freqAP, ampsAP = quickFFT_k(tClip, aAP_Clip, sampleRate, "none", dummyVariable)
            freqVert, ampsVert = quickFFT_k(tClip, aVert_Clip, sampleRate, "none", dummyVariable)
            meanAccelML   = np.mean(abs(aML_Clip))     #Mean abs acceleration in the ML direction during time window
            meanAccelAP   = np.mean(abs(aAP_Clip))     #Mean abs acceleration in the AP direction during time window
            meanAccelVert = np.mean(abs(aVert_Clip))   #Mean abs acceleration in the Vert direction during time window
            meanAmpML     = np.mean(abs(ampsML))       #Mean abs amplitude for ML data (FFT'd)
            meanAmpAP     = np.mean(abs(ampsAP))       #Mean abs amplitude for AP data (FFT'd)
            meanAmpVert   = np.mean(abs(ampsVert))     #Mean abs amplitude for Vert data (FFT'd)
            #Start Filling Arrays:
            ampSumML[tm] = np.sum(abs(ampsML))     # sum of absolute value of (FFT) amplitudes in window (ML)
            ampSumAP[tm] = np.sum(abs(ampsAP))     # sum of absolute value of (FFT) amplitudes in window (AP)
            ampSumVert[tm] = np.sum(abs(ampsVert)) # sum of absolute value of (FFT) amplitudes in window (Vert)
            maxAmpML[tm] = np.max(abs(ampsML))     # abs val of maximum amplitude in FFT'd ML data
            maxAmpAP[tm] = np.max(abs(ampsAP))     # abs val of maximum amplitude in FFT'd AP data
            maxAmpVert[tm] = np.max(abs(ampsVert)) # abs val of maximum amplitude in FFT'd vert data
            #Crudely, the number of 'peaks' in the accel data:
            for m in range(0, 2*windowHalfLength + 1):
                if aML_Clip[m] > peakThreshold*meanAccelML:
                    numPeaksML[tm] += 1.0
                if aAP_Clip[m] > peakThreshold*meanAccelAP:
                    numPeaksAP[tm] += 1.0
                if aVert_Clip[m] > peakThreshold*meanAccelVert:
                    numPeaksVert[tm] += 1.0
            maxAccelAmpML[tm] = np.max(abs(aML_Clip)) - np.min(abs(aML_Clip))       # abs(max - min accel) for ML accel data
            maxAccelAmpAP[tm] = np.max(abs(aAP_Clip)) - np.min(abs(aAP_Clip))       # abs(max - min accel) for AP accel data
            maxAccelAmpVert[tm] = np.max(abs(aVert_Clip)) - np.min(abs(aVert_Clip)) # abs(max - min accel) for Vert accel data
            rmsAmpsLowfML[tm] = getRMS(ampsML[freqML<lowfBandMax])
            rmsAmpsLowfAP[tm] = getRMS(ampsAP[freqAP<lowfBandMax])
            rmsAmpsLowfVert[tm] = getRMS(ampsVert[freqVert<lowfBandMax])
            rmsAmpsMidfML[tm] = getRMS(ampsML[(freqML>midfBandMin)*(freqML<midfBandMax)])
            rmsAmpsMidfAP[tm] = getRMS(ampsAP[(freqAP>midfBandMin)*(freqAP<midfBandMax)])
            rmsAmpsMidfVert[tm] = getRMS(ampsVert[(freqVert>midfBandMin)*(freqVert<midfBandMax)])
            rmsAmpsHighfML[tm] = getRMS(ampsML[(freqML>highfBandMin)*(freqML<highfBandMax)])
            rmsAmpsHighfAP[tm] = getRMS(ampsAP[(freqAP>highfBandMin)*(freqAP<highfBandMax)])
            rmsAmpsHighfVert[tm] = getRMS(ampsVert[(freqVert>highfBandMin)*(freqVert<highfBandMax)])
            freqOfMaxAmpML[tm] =   freqML[np.argmax(ampsML)]     # frequency of the largest (absolute) magnitude in FFT'd data (ML)
            freqOfMaxAmpAP[tm] =   freqAP[np.argmax(ampsAP)]     # frequency of the largest (absolute) magnitude in FFT'd data (AP)
            freqOfMaxAmpVert[tm] = freqVert[np.argmax(ampsVert)] # frequency of the largest (absolute) magnitude in FFT'd data (Vert)
            ratio1[tm] = rmsAmpsHighfML[tm]/rmsAmpsLowfAP[tm]      # ratio of rms high freq ML to rms low freq AP
            ratio2[tm] = rmsAmpsHighfML[tm]/rmsAmpsHighfAP[tm]     # ratio of rms high freq ML to rms high freq AP
            ratio3[tm] = maxAmpML[tm]/meanAmpAP                    # ratio of abs(maxAmpML) to average abs(AP amplitude) (FFT'd)
            ratio4[tm] = freqOfMaxAmpML[tm]/freqOfMaxAmpAP[tm]     # ratio of freqOfMaxAmpML/freqOfMaxAmpAP
            ratio5[tm] = freqOfMaxAmpVert[tm]/freqOfMaxAmpAP[tm]   # ratio of freqOfMaxAmpVert/freqOfMaxAmpAP
            ratio6[tm] = freqOfMaxAmpML[tm]/freqOfMaxAmpVert[tm]   # ratio of freqOfMaxAmpML/freqOfMaxAmpVert
        for dt in range(0, numPoints):  
            featureList[dt].append(t[dt])
            featureList[dt].append(aVert[dt])
            featureList[dt].append(aML[dt])
            featureList[dt].append(aAP[dt]) 
            featureList[dt].append(ampSumML[dt])
            featureList[dt].append(ampSumAP[dt])
            featureList[dt].append(ampSumVert[dt])
            featureList[dt].append(maxAmpML[dt])
            featureList[dt].append(maxAmpAP[dt])
            featureList[dt].append(maxAmpVert[dt])
            featureList[dt].append(numPeaksML[dt]) 
            featureList[dt].append(numPeaksAP[dt])
            featureList[dt].append(numPeaksVert[dt])
            featureList[dt].append(maxAccelAmpML[dt])
            featureList[dt].append(maxAccelAmpAP[dt])
            featureList[dt].append(maxAccelAmpVert[dt])
            featureList[dt].append(rmsAmpsLowfML[dt])
            featureList[dt].append(rmsAmpsLowfAP[dt])
            featureList[dt].append(rmsAmpsLowfVert[dt])
            featureList[dt].append(rmsAmpsMidfML[dt])
            featureList[dt].append(rmsAmpsMidfAP[dt])
            featureList[dt].append(rmsAmpsMidfVert[dt])
            featureList[dt].append(rmsAmpsHighfML[dt])
            featureList[dt].append(rmsAmpsHighfAP[dt])
            featureList[dt].append(rmsAmpsHighfVert[dt])
            featureList[dt].append(freqOfMaxAmpML[dt])
            featureList[dt].append(freqOfMaxAmpAP[dt])
            featureList[dt].append(freqOfMaxAmpVert[dt])
            featureList[dt].append(ratio1[dt]) 
            featureList[dt].append(ratio2[dt]) 
            featureList[dt].append(ratio3[dt]) 
            featureList[dt].append(ratio4[dt]) 
            featureList[dt].append(ratio5[dt]) 
            featureList[dt].append(ratio6[dt]) 
            #Finally, add the bools to the end:
            featureList[dt].append(boolStartHes[dt])
            featureList[dt].append(boolTurn[dt])
            featureList[dt].append(boolWalking[dt])
        #note to self, store all info in a list and then make dataframe from list
        #then can write df to file per https://www.kaggle.com/code/paultimothymooney/how-to-save-a-file-to-the-notebook-output-folder/notebook
        df = pd.DataFrame(featureList)
        df.columns = ["time", "accelVert", "accelML", "accelAP", 
                       "ampSumML", "ampSumAP", "ampSumVert", 
                       "maxAmpML", "maxAmpAP", "maxAmpVert", 
                       "numPeaksML", "numPeaksAP", "numPeaksVert", 
                       "maxAccelAmpML", "maxAccelAmpAP", "maxAccelAmpVert", 
                       "rmsAmpsLowfML", "rmsAmpsLowfAP", "rmsAmpsLowfVert", 
                       "rmsAmpsMidfML", "rmsAmpsMidfAP", "rmsAmpsMidfVert", 
                       "rmsAmpsHighfML", "rmsAmpsHighfAP", "rmsAmpsHighfVert", 
                       "freqOfMaxAmpML", "freqOfMaxAmpAP", "freqOfMaxAmpVert", 
                       "ratio1", "ratio2", "ratio3",
                       "ratio4", "ratio5", "ratio6",
                       "StartHesitation", "Turn", "Walking"]
        fileString = "/kaggle/working/"+filename[:-4]+"_newFeatures.csv"
        df.to_csv(fileString, index=False)
        #df.to_csv(fileString, index=False, columns=["time", "accelVert", "accelML", "accelAP", "ampSumML", "ampSumAP", "ampSumVert", "maxAmpML", "maxAmpAP", "maxAmpVert", "maxAccelAmpML", "maxAccelAmpAP", "maxAccelAmpVert", "freqOfMaxAmpML", "freqOfMaxAmpAP", "freqOfMaxAmpVert", "StartHesitation", "Turn", "Walking"])


**This code will create new csv files that can then be routed into a Kaggle data set for later use.**